In [1]:
import sys
sys.path.extend(['/Users/zsolt/Colloid_git/TractionRheoscopy'])
from data_analysis import static as da
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from importlib import reload
import numpy as np

In [2]:
hdf_stem = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/locations_stitch/'
sed_fName = 'tfrGel10212018A_shearRun10292018f_sed_stitched.h5'
gel_fName = 'tfrGel10212018A_shearRun10292018f_gel_stitched.h5'

figPath = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/plots'

In [3]:
sedPos = da.loadData2Mem(hdf_stem + sed_fName)

In [4]:
sed_strainTraj_dt3 = pd.read_hdf(hdf_stem + '/{}'.format('tfrGel10212018A_shearRun10292018f_sed_strainTraj_dt3.h5'))

## Interpolate the strain in 3D using linear nearest neighbor

In [5]:
sed_strainTraj_dt3.head()

(0,1)  (0,3)  (1,4)  (2,5)  (3,6)  (4,7)  (5,8)  (6,9)  \
particle values                                                              
0        D2_min  0.176064    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         exx     0.016755    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         exy    -0.005082    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         exz     0.009494    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         eyy    -0.010655    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                 (7,10)  (8,11)  ...  (77,80)  (78,81)  (79,82)  (80,83)  \
particle values                  ...                                       
0        D2_min     NaN     NaN  ...      NaN      NaN      NaN      NaN   
         exx        NaN     NaN  ...      NaN      NaN      NaN      NaN   
         exy        NaN     NaN  ...      NaN      NaN      NaN      NaN   
         exz        NaN     NaN  ...      NaN      NaN      NaN      NaN   
         eyy        NaN     NaN  ...      NaN      NaN      NaN      NaN   

                 (81,84)  (82,85)  (83,86)  (84,87)  (85,88)  (86,89)  
particle values                                                        
0        D2_min      NaN      NaN      NaN      NaN      NaN      NaN  
         exx         NaN      NaN      NaN      NaN      NaN      NaN  
         exy         NaN      NaN      NaN      NaN      NaN      NaN  
         exz         NaN      NaN      NaN      NaN      NaN      NaN  
         eyy         NaN      NaN      NaN      NaN      NaN      NaN  

[5 rows x 88 columns]

In [144]:
strain_exz = sed_strainTraj_dt3.loc[(slice(None),'exz'),:]['(0,1)'].unstack().dropna()
idx = strain_exz.index

In [145]:
pts = sedPos.loc[(0,slice(None))][['x (um, imageStack)', 'y (um, imageStack)', 'z (um, imageStack)']].loc[idx]

In [191]:
pts.head()

,"x (um, imageStack)","y (um, imageStack)","z (um, imageStack)"
particle,,,
0,23.480112,2.221875,27.385583
1,31.531264,2.180390,27.351286
4,35.347537,2.748365,27.335966
5,42.509473,2.716699,27.265042
6,40.966373,3.109315,27.335794


In [219]:
zz,yy,xx = np.mgrid[35:70:0.7, 10:230:0.7,10:230:0.7]

In [147]:
from scipy import interpolate

In [188]:
pts_np = np.array([pts[key].to_numpy() for key in ['z (um, imageStack)', 'y (um, imageStack)', 'x (um, imageStack)']])

In [220]:
strain_interp = interpolate.griddata(np.transpose(pts_np), strain_exz.to_numpy().squeeze(),(zz,yy,xx),method='linear')

In [174]:
strain_interp[np.isnan(strain_interp)].shape 

(0,)

In [175]:
strain_interp[~np.isnan(strain_interp)].shape

(4961250,)

## Write a correlation function with limited shift range

In [183]:
import numba
from data_analysis import eshelby_inclusion as eshelby
from importlib import reload

In [185]:
reload(eshelby)

<module 'data_analysis.eshelby_inclusion' from '/Users/zsolt/Colloid_git/TractionRheoscopy/data_analysis/eshelby_inclusion.py'>

In [221]:
strain_interp.shape

(50, 315, 315)

In [223]:
eshelby.spatialCorr(strain_interp,strain_interp,(5,5,5))

array([[[-1.93595024e-03, -3.22408985e-04,  1.96978585e-03, ...,
         -1.49639862e-03, -1.31990858e-03, -5.38960844e-04],
        [-2.52145752e-03, -1.90380913e-03, -4.33211186e-04, ...,
         -4.14035227e-03, -3.37619048e-03, -1.90165232e-03],
        [-4.87461647e-03, -6.43765434e-03, -6.38030573e-03, ...,
         -8.94246243e-03, -7.56719763e-03, -4.58299133e-03],
        ...,
        [-3.20221901e-03, -6.39717138e-03, -8.20911279e-03, ...,
         -4.43863754e-03, -1.84269540e-03,  6.05979947e-04],
        [ 5.25236409e-04, -1.17503103e-03, -2.79732200e-03, ...,
         -6.44568710e-05,  1.60022822e-03,  2.18681077e-03],
        [ 2.35074501e-03,  1.44059165e-03, -4.87729655e-06, ...,
          1.45564383e-03,  2.76868419e-03,  2.88129988e-03]],

       [[-2.51780658e-03, -2.44709234e-03, -1.09222538e-03, ...,
         -2.21821195e-03, -3.08221869e-03, -3.05173305e-03],
        [-4.77044994e-03, -7.28139844e-03, -7.46668871e-03, ...,
         -8.85297040e-03, -8.37815232e

## Test of interpolate.griddata

In [205]:
def g(x,y,z): return x*y*z
pts = np.random.rand(10000,3)
val = g(pts[:,0],pts[:,1],pts[:,2])

In [207]:
pts;

In [216]:
xx,yy,zz = np.mgrid[0:1:0.5, 0:1:0.1,0:1:0.1]

In [203]:
xx;

In [217]:
out = interpolate.griddata(pts,val,(zz,yy,xx))

In [218]:
out.shape

(2, 10, 10)